# Soal

![Soal](img/soal_1.jpg)
![Tabel Soal](img/soal_1_tabel.jpg)

Sumber: [Kode](http://www.personal.psu.edu/jpm46/pubs/ASEE_2017b.pdf)

In [10]:
import pyromat as pyro
import matplotlib.pyplot as plt
import numpy as np
steam = pyro.get('ig.H2O')


# Set the limit conditions based on the components
#Case A (losing)
p1Pa = 0.1e6
p2Pa = 8e6
T3 = 600
mdot = 5.0
syscost = 1000000
FuelEff = 0.85
eta_pump = 0.85
eta_turb = 0.85
#Case B (winning)
p1Pa = 0.05e6
p2Pa = 10e6
T3 = 650
mdot = 5.5
syscost = 2000000
FuelEff = 0.9
eta_pump = 0.88
eta_turb = 0.9
#Convert p to bar
p1 = p1Pa*1e-5
p2 = p2Pa*1e-5
#State 1 is a saturated liquid @ p1
T1 = steam.Ts(p=p1) #Temperature
h1,null = steam.hs(p=p1) #Enthalpy
s1,null = steam.ss(p=p1) #Entropy
#State 2s is isentropic from p1-p2
s2s = s1 #isentropic
#isentropic Temperature
T2s = steam.T_s(p=p2,s=s2s)
#isentropic Enthalpy
h2s = steam.h(p=p2,T=T2s)
#State 2 is found by applying the isentropic efficiency
Wps = h2s-h1 #Isentropic work
Wp = Wps/eta_pump #Actual work
h2 = Wp+h1 #Actual enthalpy at 2
T2 = steam.T_h(p=p2,h=h2) #Actual Temperature
s2 = steam.s(T=T2,p=p2) #Actual entropy
#State 3 is based on known p3 & T3
p3 = p2 #same pressure
h3 = steam.h(p=p3,T=T3) #Enthalpy
s3 = steam.s(p=p3,T=T3) #Entropy
#State 4s is isentropic from p3-p4
p4 = p1 #same as initial pressure
s4s = s3 #isentropic
#isentropic Temperature (including quality)
T4s,x4s = steam.T_s(p=p4,s=s4s,quality=True)
if x4s<0: #superheated vapor quality will be -1
    h4s = steam.h(T=T4s,p=p4) #Superheated vapor
else:
    h4s = steam.h(x=x4s,p=p4) #Liq/Vap mixture
#State 4 is found by applying the isentropic efficiency
Wts = h3-h4s #Isentropic work
Wt = Wts*eta_turb #Actual work
h4 = h3-Wt #Actual enthalpy at 4
#Actual Temperature (including quality)
T4,x4 = steam.T_h(p=p4,h=h4,quality=True)
if x4<0: #superheated vapor quality will be -1
    s4 = steam.s(T=T4,p=p4) #Superheated vapor
else:
    s4 = steam.s(x=x4,p=p4) #Liq/Vap mixture
#Find the work and heat transfer terms
Qhi = (h3-h2)
Qlo = (h4-h1)
Wnet = (Wt-Wp)
eff = Wnet/Qhi
#Print some cycle properties
if(x4>0):
    print("Turbine Outlet Quality: {:0.2f}".format(x4))
else:
    print("Turbine Outlet Quality: Superheated")
    
    print("Efficiency: {:0.2%}".format(eff))
print("Net Power: {:0.2f} kJ/kg".format(Wnet))
# Generate some diagrams
color = 'r'
marker = '.'
smarker = 'x'
# Let figure 1 be a T-s diagram
f1 = plt.figure(1)
ax1 = f1.add_subplot(111)
ax1.set_xlabel('Entropy, s (kJ/kg/K)')
ax1.set_ylabel('Temperature, T (K)')
ax1.set_title('Rankine Cycle T-s Diagram')
# Generate the dome on both plots
Tt,pt = steam.triple()
Tc,pc = steam.critical()
T = np.linspace(Tt,Tc,50)
p = steam.ps(T)
sL,sV = steam.ss(T=T,p=p)
ax1.plot(sL,T,'k')
ax1.plot(sV,T,'k')
# Process 1-2s (isentropic compression of a liquid)
s = np.linspace(s1,s2s,20)
p = np.linspace(p1,p2,20)
T = steam.T_s(p=p,s=s)
ax1.plot(s,T,color+':',linewidth=1)
ax1.plot(s1,T1,color+marker)
ax1.plot(s2s,T2s,color+smarker)
# Process 1-2 (actual compression of a liquid)
s = np.linspace(s1,s2,20)
p = np.linspace(p1,p2,20)
T = steam.T_s(p=p,s=s)
ax1.plot(s,T,color,linewidth=1.5)
ax1.plot(s2,T2,color+marker)
#process 2-3 (heat add)
s = np.linspace(s2,s3,200)
p = p2*np.ones(s.shape)
T = steam.T_s(p=p,s=s)
ax1.plot(s,T,color,linewidth=1.5)
ax1.plot(s3,T3,color+marker)
# Process 3-4s (isentropic compression of a liquid)
s = np.linspace(s3,s4s,20)
p = np.linspace(p3,p4,20)
T = steam.T_s(p=p,s=s)
ax1.plot(s,T,color+':',linewidth=1)
ax1.plot(s4s,T4s,color+smarker)
#process 3-4 (actual expansion)
s = np.linspace(s3,s4,20)
p = np.linspace(p3,p4,20)
T = steam.T_s(p=p,s=s)
ax1.plot(s,T,color,linewidth=1.5)
ax1.plot(s4,T4,color+marker)
#process 4-1 (heat rej)
s = np.linspace(s4,s1,40)
p = p4*np.ones(s.shape)
T = steam.T_s(p=p,s=s)
ax1.plot(s,T,color,linewidth=1.5)
ax1.grid('on')
ax1.set_ylim([300,800])
f1.show()
#Pause
input("Press Enter to continue...")

AttributeError: 'ig2' object has no attribute 'Ts'